# 函数装饰器和闭包

## 装饰器基础概念

装饰器是可调用的对象，其要求传入另一个函数。装饰器可能会处理被装饰的函数或者将被装饰的函数替换为另一个函数。

装饰器从行为上来说和一般的将函数作为参数的可调用对象没有本质上的区别，在某些情况下使用装饰器可能更方便。但是装饰器具有独特优势：在加载模块时立即执行。

在加载时立即执行是装饰器的一大优势，这样就能实现在运行时改变程序的行为或者改变内部状态。registration.py例子展示了装饰器在导入模块时立即执行这一特性。

registration.py中的装饰器并没有改变被装饰的函数的行为，但是该装饰器将被装饰的函数计入一个list，则在有些应用场景中是有好处的，例如在“策略”模式中自动获取所有可选择的具体模式。

In [1]:
def deco(func):

    def inner():
        print("running inner()")
    return inner

@deco
def target():
    print("running target()")

target()
target

running inner()


<function __main__.deco.<locals>.inner()>

In [10]:
import os

print(os.popen("python ./registration.py").read())

import registration

print(registration.registry)

running register<function f1 at 0x0000025031700708>
running register<function f2 at 0x00000250317001F8>
running main()
registry -> [<function f1 at 0x0000025031700708>, <function f2 at 0x00000250317001F8>]
running f1()
running f2()
running f3()

[<function f1 at 0x00000200059719D8>, <function f2 at 0x0000020005971B88>]


## Python中的变量作用域

装饰器的使用有时候涉及到闭包的概念，而闭包的概念与Python中变量作用域息息相关

下述展示的两个函数f1和f2，其中f1使用了未声明的变量b，毫无疑问会报错；而f2同样也会报错，并且是在print(b)位置报错。从字节码可以分析上述两个函数报错的原因：

对于f1，字节码显示f1执行了如下操作：
1. 加载全局名称print
2. 加载局部变量a(字节码：LOAD_FAST)
3. 调用函数，执行print
4. 删除栈顶项
5. 加载全局名称print
6. 加载全局变量b(字节码：LOAD_GLOBAL)
7. 调用函数，执行print
8. 删除栈顶项

对于f2，字节码显示f2执行了如下操作：
1. 加载全局名称print
2. 加载局部变量a(字节码：LOAD_FAST)
3. 调用函数，执行print
4. 删除栈顶项
5. 加载全局名称print
6. 加载局部变量b(字节码：LOAD_FAST)
7. 调用函数，执行print
8. 删除栈顶项

可以看到f1和f2唯一不同的是对变量b的加载。f1由于没有在函数中声明变量b，因此主动使用LOAD_GLOBAL尝试从全局中加载b；f2则将变量b判断为局部变量，并且使用LOAD_FAST尝试从本地环境中加载b。

f1和f2唯一的不同是是否有赋值语句。f2对变量b有赋值，并且没有global声明，因此变量b在f2中被自动判断为局部变量，并尝试在print(b)中从本地环境中获取变量b的值。

若对更多的变量类型进行测试会发现：一些类型在函数定义体中进行赋值后会尝试使用LOAD_FAST调用该变量，而另一些类型在函数定义提中进行赋值后依然会尝试使用LOAD_GLOBAL调用该变量。就测试结果来说，若是不可变类型（tuple、str等）使用赋值语句后会尝试从本地环境中获取该变量的值，若是可变类型（list、array、dict、set）使用赋值语句后依然会尝试从全局环境中获取该变量的值。对于不可变类型，若想使用全局变量，则可以使用global声明

In [11]:
from dis import dis

def f1(a):
    print(a)
    print(b)

dis(f1)

b=1
def f2(a):
    print(a)
    print(b)
    b=9

dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
 11           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

 12           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

 13          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## 闭包（Closure）

闭包和自由变量（free variable）息息相关。

Python允许函数中嵌套函数，并且内层的函数能够访问外层函数的局部变量，但是外层函数无法访问内层函数的局部变量。这种结构实际上形成了闭包，当外层函数对应的资源被释放后，内层函数依然能够访问到外层函数的局部变量，即这些局部变量以某种形式留存下来。

下述例子中，series是make_averager的局部变量，该变量用于保存输入的价格；averager则是嵌套于make_averager中的函数，其获取输入的价格并且将该价格存入series然后计算均值。

这一例子中，make_averager会返回其内嵌的函数averager，正常来说，make_averager在返回的那一刻，其占用的资源会被释放，series作为该函数的局部变量也应当被释放。但是从运行结果来看，series不仅没有被释放，而且能正常记录输入的数据。

上述功能能正常执行实际上依赖的是自由变量。从反编译的字节码也可以看出，不同于上述例子中的LOAD_GLOBAL和LOAD_FAST，这里加载series时使用的是LOAD_DEREF。实际上，series围在本地作用域中绑定，而是以自由变量的形式存在，其实际上绑定在__closure__属性中，并且可以从__code__.co_freevars中查询到这些自由变量的名称。

闭包实际上实现了在一个局部变量的定义作用域不再可用时，其依然可以以自由变量的形式存在于闭包中。


值得注意的是，这里实际上也存在类似于上一个例子中全局变量与局部变量的问题。在闭包中，可变类型的变量无论是读取还是赋值均采用LOAD_DEREF加载这些变量；而不可变类型对应的变量若使用了赋值语句则会直接变为尝试使用LOAD_FAST加载这些变量，这在一些应用中会导致错误——变量没有声明。同样的，类似于global声明使得程序始终使用LOAD_GLOBAL加载变量，nonlocal声明则可以使得程序始终使用LOAD_DEREF加载变量。

In [17]:
def make_averager():

    series = list()

    def averager(new_value):

        series.append(new_value)
        return sum(series) / len(series)
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

dis(avg)


10.0
10.5
11.0
  7           0 LOAD_DEREF               0 (series)
              2 LOAD_METHOD              0 (append)
              4 LOAD_FAST                0 (new_value)
              6 CALL_METHOD              1
              8 POP_TOP

  8          10 LOAD_GLOBAL              1 (sum)
             12 LOAD_DEREF               0 (series)
             14 CALL_FUNCTION            1
             16 LOAD_GLOBAL              2 (len)
             18 LOAD_DEREF               0 (series)
             20 CALL_FUNCTION            1
             22 BINARY_TRUE_DIVIDE
             24 RETURN_VALUE


In [18]:
def make_averager_2():

    total = 0
    count = 0

    def averager(new_value):

        total += new_value
        count += 1
        return total / count
    
    return averager

avg_2 = make_averager_2()
dis(avg_2)

  8           0 LOAD_FAST                1 (total)
              2 LOAD_FAST                0 (new_value)
              4 INPLACE_ADD
              6 STORE_FAST               1 (total)

  9           8 LOAD_FAST                2 (count)
             10 LOAD_CONST               1 (1)
             12 INPLACE_ADD
             14 STORE_FAST               2 (count)

 10          16 LOAD_FAST                1 (total)
             18 LOAD_FAST                2 (count)
             20 BINARY_TRUE_DIVIDE
             22 RETURN_VALUE
